# Sabermetrics - Code

The data set here is a data set of pitcher information combined with predictions on the IP (or _innings pitched_) and RA9 (or _runs allowed per 9 innings_). These predictions come from four different groups who all have different methods of calculating their predictions.

We have two data sets `train_saber.csv` and `test_saber.csv`. `train_saber.csv` are predictions for the 2012 season, which has already happened for Billy Beane, so we also have the realizations for IP and RA9 for the 2012 season. We are trying to predict the 2013 season. `test_saber.csv` has all of the currently available information that Billy Beane would have in making his hiring decisions, but it obviously does not have any data from the 2013 season.

Our job is to take a $15M budget and find two free agent pitchers to round out our starting rotation. We'll also assume that we're not trying to find the next phenom ace, but will be happy with two workhorse guys to go out, chew up some innings, and give good quality starts.

## Import Packages and Data

In [279]:
import numpy as np # Library for math operations
import pandas as pd # Library for data handling
import sklearn # The machine learning library we will be using in this entire course
from sklearn import tree # Tree function is used for visualizing decision tree
from sklearn.metrics import * # Importing function that can be used to calculate different metrics
from sklearn.tree import DecisionTreeClassifier # Importing Decision Tree Classifier 
from sklearn.ensemble import RandomForestClassifier  # Importing Random Forest Classifier 
from sklearn.model_selection import train_test_split # Importing function that can split a dataset into training and testing set
from sklearn.preprocessing import MinMaxScaler # Importing function for scaling the data
from sklearn.preprocessing import LabelEncoder # Importing function for processing the labels
from sklearn.ensemble import GradientBoostingClassifier # Importing GB Classifier
from sklearn.model_selection import GridSearchCV # Importing GridSearchCV
from sklearn.model_selection import RandomizedSearchCV # Importing RandomSearchCV
import xgboost as xgb
from xgboost import XGBClassifier # Importing the XGBoost Classifier 
from xgboost import XGBRegressor # Importing the XGBoost Regressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error, make_scorer
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV

import matplotlib.pyplot as plt # Importing the package for plotting
plt.style.use('fivethirtyeight') # Use the styling from FiveThirtyEight Website
import seaborn as sns # Importing another package for plotting

In [621]:
train_df = pd.read_csv("train_saber.csv")

In [622]:
test_df = pd.read_csv("test_saber.csv")

In [82]:
display(test_df.head(3))

,NAME,IP_fans,RA9_fans,IP_marcel,RA9_marcel,IP_rotochamp,RA9_rotochamp,IP_zips,RA9_zips,playerID,...,bats,throws,debut,finalGame,retroID,bbrefID,deathDate,birthDate,IP,RA9
0,Aaron Crow,66,3.538949,63.7,3.843843,60,3.593394,64.0,4.137848,crowaa01,...,R,R,2011-03-31,NaN,crowa001,crowaa01,NaN,1986-11-10,NaN,NaN
1,Adam Wainwright,215,3.462725,159.3,3.996290,195,3.462725,173.7,3.724063,wainwad01,...,R,R,2005-09-11,NaN,waina001,wainwad01,NaN,1981-08-30,NaN,NaN
2,Addison Reed,61,3.789397,53.3,4.595189,60,4.246739,66.7,4.257628,reedad01,...,L,R,2011-09-04,NaN,reeda001,reedad01,NaN,1988-12-27,NaN,NaN


In [8]:
display(test_df.head(3))

,NAME,IP_fans,RA9_fans,IP_marcel,RA9_marcel,IP_rotochamp,RA9_rotochamp,IP_zips,RA9_zips,playerID,...,bats,throws,debut,finalGame,retroID,bbrefID,deathDate,birthDate,IP,RA9
0,Aaron Crow,66,3.538949,63.7,3.843843,60,3.593394,64.0,4.137848,crowaa01,...,R,R,2011-03-31,9/28/2014,crowa001,crowaa01,NaN,1986-11-10,48.000000,3.562500
1,Adam Wainwright,215,3.462725,159.3,3.996290,195,3.462725,173.7,3.724063,wainwad01,...,R,R,2005-09-11,9/22/2014,waina001,wainwad01,NaN,1981-08-30,241.666667,3.091034
2,Addison Reed,61,3.789397,53.3,4.595189,60,4.246739,66.7,4.257628,reedad01,...,L,R,2011-09-04,9/27/2014,reeda001,reedad01,NaN,1988-12-27,71.333333,3.911215


In [9]:
def summarize_dataframe(df):
    """Summarize a dataframe, and report missing values."""
    missing_values = pd.concat([pd.DataFrame(df.columns, columns=['Variable Name']), 
                      pd.DataFrame(df.dtypes.values.reshape([-1,1]), columns=['Data Type']),
                      pd.DataFrame(df.isnull().sum().values, columns=['Missing Values']), 
                      pd.DataFrame([df[name].nunique() for name in df.columns], columns=['Unique Values'])], 
                     axis=1).set_index('Variable Name')
    with pd.option_context("display.max_rows", 1000):
        display(pd.concat([missing_values, df.describe(include='all').transpose()], axis=1).fillna(""))

In [557]:
summarize_dataframe(X_train)

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
BAOpp,float64,0,70,119.0,,,,0.259176,0.032048,0.171,0.24,0.256,0.273,0.376
BB,int64,0,64,119.0,,,,44.495798,22.122109,1.0,29.0,46.0,58.0,105.0
BFP,int64,0,110,119.0,,,,607.134454,262.689058,24.0,425.5,728.0,823.5,956.0
BK,int64,0,5,119.0,,,,0.495798,0.811715,0.0,0.0,0.0,1.0,4.0
CG,int64,0,7,119.0,,,,0.806723,1.202239,0.0,0.0,0.0,1.0,6.0
ER,int64,0,69,119.0,,,,64.352941,27.681172,3.0,42.0,70.0,82.5,118.0
ERA,float64,0,98,119.0,,,,4.218403,1.203247,1.67,3.385,4.02,4.85,8.28
G,int64,0,34,119.0,,,,24.588235,9.548637,1.0,18.5,29.0,32.0,38.0
GF,int64,0,9,119.0,,,,0.394958,1.530584,0.0,0.0,0.0,0.0,10.0
GS,int64,0,32,119.0,,,,23.428571,9.814009,1.0,16.0,28.0,32.0,34.0


In [88]:
summarize_dataframe(test_df)

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
NAME,object,0,119,119.0,119,Aaron Crow,1,,,,,,,
IP_fans,int64,0,80,119.0,,,,147.369748,59.680202,51.0,72.0,166.0,198.0,233.0
RA9_fans,float64,0,88,119.0,,,,3.86434,0.649077,2.450041,3.413724,3.800287,4.388296,5.128753
IP_marcel,float64,0,99,119.0,,,,127.807563,52.345926,34.0,66.15,146.0,173.5,204.3
RA9_marcel,float64,0,89,119.0,,,,4.046251,0.561659,2.754935,3.609728,4.061624,4.46452,5.477204
IP_rotochamp,int64,0,30,119.0,,,,137.647059,61.157007,25.0,65.0,160.0,195.0,220.0
RA9_rotochamp,float64,0,87,119.0,,,,3.918694,0.718133,1.676917,3.430058,3.920066,4.437297,5.488093
IP_zips,float64,0,105,119.0,,,,135.678992,54.751598,23.0,71.35,150.7,174.7,228.3
RA9_zips,float64,0,98,119.0,,,,4.161456,0.782862,1.709584,3.609728,4.148737,4.70408,6.184994
playerID,object,0,119,119.0,119,crowaa01,1,,,,,,,


## Separate Out Starters from Closers

Since we only want to acquire starters, let's weed out closing pitchers from the dataset (pitchers who enter games in the 8th or 9th inning to get the last few batters out). This should improve our model's performance in predicting starter IP and RA9 since the trends between these groups are so different. We'll save discussions of teams using 'openers' for a different day...

In [623]:
# Create an Innings Pitched Per Game Feature
train_df['IPperG'] = train_df['IP'] / train_df['G']
train_df['IPperG_fans'] = train_df['IP_fans'] / train_df['G']
train_df['IPperG_marcel'] = train_df['IP_marcel'] / train_df['G']
train_df['IPperG_rotochamp'] = train_df['IP_rotochamp'] / train_df['G']
train_df['IPperG_zips'] = train_df['IP_zips'] / train_df['G']

In [624]:
# Differentiate Starters as pitching more than a certain number of innings per games on avg.
# We'll use 3 to exclude closers and leave the max number of pitchers available to us
minIP = 3

train_df['Starter'] = train_df['IPperG'].apply(lambda x: 1 if x >= minIP else 0)
train_df['Starter_fans'] = train_df['IPperG_fans'].apply(lambda x: 1 if x >= minIP else 0)
train_df['Starter_marcel'] = train_df['IPperG_marcel'].apply(lambda x: 1 if x >= minIP else 0)
train_df['Starter_rotochamp'] = train_df['IPperG_rotochamp'].apply(lambda x: 1 if x >= minIP else 0)
train_df['Starter_zips'] = train_df['IPperG_zips'].apply(lambda x: 1 if x >= minIP else 0)

In [625]:
# There aren't any games played in the 2013 dataset, so to get a rough categorization, we'll assume next year's IP will mirror last year's. 
# We may miss middle-reliever ready to move up or a pitcher coming back from an injury, but I think that's alright given our needs.
test_df = pd.merge(test_df, train_df[['playerID','G']], on="playerID", suffixes=('', '_12'))

In [626]:
# Calculate IP/G on test data
test_df['IPperG_fans'] = test_df['IP_fans'] / test_df['G_12']
test_df['IPperG_marcel'] = test_df['IP_marcel'] / test_df['G_12']
test_df['IPperG_rotochamp'] = test_df['IP_rotochamp'] / test_df['G_12']
test_df['IPperG_zips'] = test_df['IP_zips'] / test_df['G_12']

# Find starters in test data
test_df['Starter_fans'] = test_df['IPperG_fans'].apply(lambda x: 'True' if x >= minIP else 'False')
test_df['Starter_marcel'] = test_df['IPperG_marcel'].apply(lambda x: 'True' if x >= minIP else 'False')
test_df['Starter_rotochamp'] = test_df['IPperG_rotochamp'].apply(lambda x: 'True' if x >= minIP else 'False')
test_df['Starter_zips'] = test_df['IPperG_zips'].apply(lambda x: 'True' if x >= minIP else 'False')

In [627]:
# Take the widest possible prediction of who may be a starter in test data, if any models predict a starter we include them
test_df.loc[(test_df['Starter_fans'] == 'True') | (test_df['Starter_marcel'] == 'True') | (test_df['Starter_rotochamp'] == 'True') | (test_df['Starter_zips'] == 'True'), 'Starter'] = 1  
test_df.loc[(test_df['Starter_fans'] != 'True') & (test_df['Starter_marcel'] != 'True') & (test_df['Starter_rotochamp'] != 'True') & (test_df['Starter_zips'] != 'True'), 'Starter'] = 0

In [628]:
# Create new train/test datasets with only our predicted starters
starter_train_df = train_df.loc[(train_df['Starter']==1)]
starter_test_df = test_df.loc[(test_df['Starter']==1)]

In [540]:
display(starter_train_df.shape)
display(starter_train_df.head(5))

(119, 76)

,NAME,IP_fans,RA9_fans,IP_marcel,RA9_marcel,IP_rotochamp,RA9_rotochamp,IP_zips,RA9_zips,playerID,...,IPperG,IPperG_fans,IPperG_marcel,IPperG_rotochamp,IPperG_zips,Starter,Starter_fans,Starter_marcel,Starter_rotochamp,Starter_zips
1,Aaron Harang,164,4.602863,156,4.701261,180,4.646595,133.7,4.712194,haranaa01,...,5.795699,5.290323,5.032258,5.806452,4.312903,True,True,True,True,True
2,Adam Wainwright,196,3.443947,83,3.203417,175,3.771942,173.3,3.411148,wainwad01,...,6.208333,6.125000,2.593750,5.468750,5.415625,True,True,False,True,True
3,Alex Cobb,103,4.099937,86,4.176469,110,4.209269,122.0,4.351400,cobbal01,...,5.927536,4.478261,3.739130,4.782609,5.304348,True,True,True,True,True
10,Barry Zito,135,5.138588,103,4.537264,155,4.887125,124.0,4.679395,zitoba01,...,5.760417,4.218750,3.218750,4.843750,3.875000,True,True,True,True,True
11,Bartolo Colon,129,4.810593,139,4.668462,150,4.001538,112.7,4.624729,colonba01,...,6.347222,5.375000,5.791667,6.250000,4.695833,True,True,True,True,True


In [541]:
display(starter_test_df.shape)
display(starter_test_df.head(5))

(87, 76)

,NAME,IP_fans,RA9_fans,IP_marcel,RA9_marcel,IP_rotochamp,RA9_rotochamp,IP_zips,RA9_zips,playerID,...,G_12,IPperG_fans,IPperG_marcel,IPperG_rotochamp,IPperG_zips,Starter_fans,Starter_marcel,Starter_rotochamp,Starter_zips,Starter
1,Adam Wainwright,215,3.462725,159.3,3.996290,195,3.462725,173.7,3.724063,wainwad01,...,32,6.718750,4.978125,6.093750,5.428125,True,True,True,True,True
2,Alex Cobb,164,4.159626,133.7,4.181404,140,3.920066,149.3,4.529854,cobbal01,...,23,7.130435,5.813043,6.086957,6.491304,True,True,True,True,True
5,Andrew Cashner,130,3.985401,53.0,4.442742,70,3.636950,99.3,4.148737,cashnan01,...,33,3.939394,1.606061,2.121212,3.009091,True,False,False,True,True
8,Barry Zito,188,4.671412,155.7,4.725858,155,4.998085,135.7,4.693191,zitoba01,...,32,5.875000,4.865625,4.843750,4.240625,True,True,True,True,True
9,Bartolo Colon,135,4.322962,151.0,4.214071,120,4.410075,127.0,4.475409,colonba01,...,24,5.625000,6.291667,5.000000,5.291667,True,True,True,True,True


## Explore Feature Importance

In [ ]:
print(log_reg.summary())

In [ ]:
import statsmodels.api as sm

#define response variable
y = y_train_ip

#define predictor variables
x = X_train

#add constant to predictor variables
x = sm.add_constant(x)

#fit regression model
model = sm.OLS(y, x).fit()

#view summary of model fit
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                     IP   R-squared:                       0.577
Model:                            OLS   Adj. R-squared:                  0.442
Method:                 Least Squares   F-statistic:                     4.252
Date:                Thu, 22 Jun 2023   Prob (F-statistic):           4.49e-11
Time:                        19:23:26   Log-Likelihood:                -948.25
No. Observations:                 182   AIC:                             1987.
Df Residuals:                     137   BIC:                             2131.
Df Model:                          44                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
IP_fans           0.3924      0.317      1.239

BorutaShap analysis can help us narrow down which IVs are contributing most to predictions of IP and RA9 and point to potential features to be created in predicting our DVs.

In [25]:
# !pip install BorutaShap

  Using cached BorutaShap-1.0.16-py3-none-any.whl (13 kB)


In [26]:
from BorutaShap import BorutaShap

# If no model is selected default is the Random Forest
# If classification is True it is a classification problem
Feature_Selector = BorutaShap(importance_measure='shap', classification=False)

### Data Prep

In [660]:
# Drop columns from the data that are either blank, don't influence our metrics of interest, or can't be calculated in 2013
drop_list = ['NAME',
            'playerID',
            'nameFirst',
            'nameLast',
            'nameGiven',
            'retroID',
            'bbrefID',
            'GIDP', 
            'debut',
             'stint',
            'birthDate',
            'birthCountry',
            'birthCity',
            'birthDay',
            'birthMonth',
            'finalGame',
            'birthState',
            'deathYear',
            'deathMonth',
            'deathDay',
            'deathCountry',
            'deathCity',
            'deathState',
            'deathDate',
            'Starter_fans',
            'Starter_zips',
            'Starter_marcel',
            'Starter_rotochamp',
            'IPperG_fans',
            'IPperG_marcel',
            'IPperG_zips',
            'IPperG_rotochamp',
             'yearID',
             'Starter',
             'teamID',
            'lgID',
            'bats',
            'throws'
            ]

X_train = starter_train_df.drop(drop_list,axis= 1)
X_train.drop(['IP','RA9','IPperG'],axis= 1,inplace=True) # Drop our DVs

y_train_IP = starter_train_df['IP'] # Create on DV for Innings Pitched
y_train_RA9 = starter_train_df['RA9'] # ...and another for Runs per 9 Innings

X_test = starter_test_df.drop(drop_list,axis= 1)
X_test.drop(['IP','RA9','G_12'],axis= 1,inplace=True) # Drop our DVs

y_test_ip = starter_test_df['IP']
y_test_ra9 = starter_test_df['RA9']

In [648]:
X_train.reset_index(drop=True, inplace=True)
y_train_ip.reset_index(drop=True, inplace=True)
y_train_ra9.reset_index(drop=True, inplace=True)

X_test.reset_index(drop=True, inplace=True)
y_test_ip.reset_index(drop=True, inplace=True)
y_test_ra9.reset_index(drop=True, inplace=True)

In [649]:
print(X_train.shape)
print(y_train_ip.shape)
print(y_train_ra9.shape)

print(X_test.shape)
print(y_test_ip.shape)
print(y_test_ra9.shape)

(119, 35)
(119,)
(119,)
(87, 35)
(87,)
(87,)


In [519]:
# Create a list for one hot encoding
enc_list = ['teamID',
'lgID',
'bats',
'throws']

In [229]:
def encode_and_bind(original_dataframe, enc_list):
    dummies = pd.get_dummies(data=original_dataframe, columns=enc_list)
    res = original_dataframe.merge(dummies, how='left')
    res = res.drop(enc_list, axis=1)
    return(res)

In [612]:
res_train = encode_and_bind(X_train, enc_list)
res_test = encode_and_bind(X_test, enc_list)

In [613]:
X_train_cat = X_train
X_test_cat = X_test

X_train = res_train
X_test = res_test

In [663]:
print(X_train.shape)
print(X_test.shape)

(119, 11)
(87, 11)


In [641]:
# Our number of columns aren't matching up because there are no Diamondbacks in the training data...
a = X_train.columns.difference(X_test.columns)
print (a)

Index([], dtype='object')


In [640]:
# So we'll add a column to reflect that
X_test['teamID_COL'] = 0

In [575]:
# Then alphabetize the columns so they match up
X_train = X_train.sort_index(axis=1)
X_test = X_test.sort_index(axis=1)

In [662]:
print(X_train.columns.tolist())

['IP_fans', 'RA9_fans', 'IP_marcel', 'RA9_marcel', 'IP_rotochamp', 'RA9_rotochamp', 'IP_zips', 'RA9_zips', 'birthYear', 'weight', 'height']


### IP

In [238]:
# What is most important in predicting IP?
Feature_Selector.fit(X=X_train, y=y_train_ip, n_trials=1000, random_state=0)

  0%|          | 0/1000 [00:00<?, ?it/s]

16 attributes confirmed important: ['HR', 'L', 'R', 'W', 'GS', 'IPperG_fans', 'G', 'SO', 'H', 'ER', 'IPperG_rotochamp', 'BB', 'IPperG_marcel', 'BFP', 'IPperG_zips', 'IPouts']
62 attributes confirmed unimportant: ['IP_marcel', 'SF', 'teamID_MIN', 'teamID_COL', 'teamID_TEX', 'RA9_fans', 'RA9_zips', 'height', 'RA9_marcel', 'teamID_SLN', 'BAOpp', 'BK', 'teamID_LAN', 'teamID_WAS', 'teamID_DET', 'SHO', 'CG', 'GF', 'birthYear', 'teamID_LAA', 'weight', 'teamID_BAL', 'teamID_KCA', 'bats_R', 'SH', 'teamID_CHA', 'IP_fans', 'yearID', 'teamID_SFN', 'bats_B', 'teamID_CLE', 'teamID_ARI', 'throws_R', 'teamID_TBA', 'stint', 'teamID_ATL', 'ERA', 'teamID_MIA', 'IP_rotochamp', 'IP_zips', 'SV', 'teamID_MIL', 'teamID_SDN', 'teamID_TOR', 'teamID_OAK', 'bats_L', 'RA9_rotochamp', 'teamID_BOS', 'teamID_NYN', 'teamID_NYA', 'lgID_NL', 'teamID_CHN', 'teamID_CIN', 'HBP', 'teamID_PIT', 'teamID_PHI', 'IBB', 'teamID_HOU', 'throws_L', 'lgID_AL', 'teamID_SEA', 'WP']
0 tentative attributes remains: []


In [240]:
# Create a features list of BShap importance, and comment out anything we don't have to use in test data
bshap_feats_ip = [
                  # 'HR', #home runs
                  # 'L', #losses
                  # 'R', #runs
                  # 'W', #wins
                  # 'GS', #games started
                  'IPperG_fans', 
                  # 'G', #games
                  # 'SO', #strike outs
                  # 'H', #hits
                  # 'ER', #earned runs
                  'IPperG_rotochamp', 
                  # 'BB', #base on balls
                  'IPperG_marcel', 
                  'BFP', #batters faced by pitcher
                  'IPperG_zips'
                  # 'IPouts' #outs pitched (IP x 3)
                 ]

In [420]:
# We don't have a BFP stat in test data, but we can make one
# IP is 3 outs, which is 3 batters, and RA9 divided by 9 is runs per inning. Each run started as a batter. 
# We won't count runners left stranded on base
# We'll watch for colinearity here but it'll be interesting to run some models with

X_train['BFP_fans'] = (X_train['RA9_fans']/9) + (X_train['IP_fans']*3)
X_train['BFP_marcel'] = (X_train['RA9_marcel']/9) + (X_train['IP_marcel']*3)
X_train['BFP_zips'] = (X_train['RA9_zips']/9) + (X_train['IP_zips']*3)
X_train['BFP_rotochamp'] = (X_train['RA9_rotochamp']/9) + (X_train['IP_rotochamp']*3)

X_test['BFP_fans'] = (X_test['RA9_fans']/9) + (X_test['IP_fans']*3)
X_test['BFP_marcel'] = (X_test['RA9_marcel']/9) + (X_test['IP_marcel']*3)
X_test['BFP_zips'] = (X_test['RA9_zips']/9) + (X_test['IP_zips']*3)
X_test['BFP_rotochamp'] = (X_test['RA9_rotochamp']/9) + (X_test['IP_rotochamp']*3)

### RA9

In [239]:
# What is most important in predicting RA9?
Feature_Selector.fit(X=X_train, y=y_train_ra9, n_trials=1000, random_state=0)

  0%|          | 0/1000 [00:00<?, ?it/s]

4 attributes confirmed important: ['ERA', 'BAOpp', 'IPperG_rotochamp', 'W']
74 attributes confirmed unimportant: ['IP_marcel', 'SF', 'teamID_COL', 'RA9_fans', 'height', 'teamID_SLN', 'teamID_LAN', 'GS', 'teamID_DET', 'G', 'CG', 'GF', 'SO', 'birthYear', 'weight', 'teamID_BAL', 'teamID_KCA', 'SH', 'IP_fans', 'throws_R', 'teamID_ARI', 'teamID_TBA', 'teamID_SDN', 'SV', 'teamID_NYA', 'teamID_PIT', 'throws_L', 'teamID_MIN', 'teamID_TEX', 'RA9_zips', 'RA9_marcel', 'BK', 'teamID_WAS', 'WP', 'SHO', 'teamID_LAA', 'ER', 'BB', 'bats_R', 'IPperG_zips', 'yearID', 'HR', 'teamID_CHA', 'IPperG_marcel', 'L', 'teamID_SFN', 'bats_B', 'teamID_CLE', 'teamID_ATL', 'H', 'teamID_MIA', 'IP_rotochamp', 'IP_zips', 'teamID_MIL', 'teamID_TOR', 'BFP', 'teamID_OAK', 'bats_L', 'IPouts', 'RA9_rotochamp', 'teamID_BOS', 'teamID_NYN', 'R', 'lgID_NL', 'teamID_CHN', 'teamID_CIN', 'HBP', 'teamID_PHI', 'IBB', 'teamID_HOU', 'lgID_AL', 'IPperG_fans', 'teamID_SEA', 'stint']
0 tentative attributes remains: []


## Train the Regressor

In [246]:
summarize_dataframe(X_test)

,Data Type,Missing Values,Unique Values,count,mean,std,min,25%,50%,75%,max
BAOpp,float64,87,0,0.0,,,,,,,
BB,float64,87,0,0.0,,,,,,,
BFP,float64,87,0,0.0,,,,,,,
BK,float64,87,0,0.0,,,,,,,
CG,float64,87,0,0.0,,,,,,,
ER,float64,87,0,0.0,,,,,,,
ERA,float64,87,0,0.0,,,,,,,
G,float64,87,0,0.0,,,,,,,
GF,float64,87,0,0.0,,,,,,,
GS,float64,87,0,0.0,,,,,,,


In [661]:
# Test data doesn't yet include player stats, so we can only train the models on data shared by both train and test.
pred_drop_list = ['BAOpp', 'BB', 'BFP', 'BK', 'CG', 'ER', 'ERA', 'G', 'GF', 'GS', 'H', 'HBP', 'HR', 'IBB','IPouts','L', 'R', 'SF', 'SH', 'SHO', 'SO', 'SV', 'W', 'WP'
               ]

X_train.drop(pred_drop_list,axis= 1,inplace=True)
X_test.drop(pred_drop_list,axis= 1,inplace=True)

In [652]:
# Then alphabetize the columns so they match up
X_train = X_train.sort_index(axis=1)
X_test = X_test.sort_index(axis=1)

In [315]:
def RMSLE(y_true, y_pred):
    y_pred_use = y_pred.clip(0)
    rmsle = mean_squared_log_error(y_true, y_pred_use)**(1/2)
    return rmsle

def accuracy(y_true, y_pred):
    """Function that returns a table showing RMSE and MAE."""
    acc_df = pd.DataFrame(data = {"RMSE": [mean_squared_error(y_true, y_pred)**(1/2)],
                                  "MAE": [mean_absolute_error(y_true, y_pred)],
                                  "R^2": [r2_score(y_true, y_pred)],
                                  "RMSLE": [RMSLE(y_true, y_pred)]})
    display(acc_df.style.hide(axis='index'))

In [664]:
#Test and Train for IP
X_train, X_valid, y_train_ra9, y_valid_ra9 = train_test_split(X_train, y_train_RA9, test_size = 0.25, random_state = 201)

### Train Regressor w/ Hyperopt Tuning

In [318]:
# !pip install hyperopt

  Using cached hyperopt-0.2.7-py2.py3-none-any.whl (1.6 MB)
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)


In [319]:
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK

#Import 'scope' from hyperopt in order to obtain int values for certain hyperparameters.
from hyperopt.pyll.base import scope

#Define the space over which hyperopt will search for optimal hyperparameters.
space = {'max_depth': scope.int(hp.randint("max_depth", 1, 5)),
        'gamma': hp.uniform ('gamma', 0,1),
        'reg_alpha' : hp.uniform('reg_alpha', 0,50),
        'reg_lambda' : hp.uniform('reg_lambda', 10,100),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0,1),
        'min_child_weight' : hp.uniform('min_child_weight', 0, 5),
        'n_estimators': hp.randint('n_estimators', 100, 1000),
        'learning_rate': hp.uniform('learning_rate', 0, .15),
        'random_state': 5,
        'max_bin' : scope.int(hp.quniform('max_bin', 200, 550, 1))}

In [665]:
y_train = y_train_ra9
y_valid = y_valid_ra9

#Define the hyperopt objective.
def hyperparameter_tuning(space):
    model = xgb.XGBRegressor(**space)
    
    #Define evaluation datasets.
    evaluation = [(X_train, y_train), (X_valid, y_valid)]
    
    #Fit the model. Define evaluation sets, early_stopping_rounds, and eval_metric.
    model.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="rmse",
            early_stopping_rounds=100,verbose=False)

    #Obtain prediction and rmse score.
    pred = model.predict(X_valid)
    rmse = mean_squared_error(y_valid, pred, squared=False)
    print ("SCORE:", rmse)
    
    #Specify what the loss is for each model.
    return {'loss':rmse, 'status': STATUS_OK, 'model': model}


In [666]:
#Run 100 trials.
trials = Trials()
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials
           )

print(best)

SCORE:                                                 
0.9753643996531596                                     
SCORE:                                                 
1.0485322378444981                                                               
SCORE:                                                                           
1.089866588786647                                                                
SCORE:                                                                           
0.9807709775290762                                                               
SCORE:                                                                           
1.0234283616122022                                                               
SCORE:                                                                           
0.9999038062399969                                                               
SCORE:                                                                           
1.0295442114

In [667]:
#Create instace of best model.
best_model = trials.results[np.argmin([r['loss'] for r in 
    trials.results])]['model']

#Examine model hyperparameters
print(best_model)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1,
             colsample_bytree=0.4960933598071299, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.26722511300906576, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.09866191748756506, max_bin=435,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=1, max_leaves=0, min_child_weight=1.5626244248189403,
             missing=nan, monotone_constraints='()', n_estimators=217, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=5, ...)


### Evaluate Model Performance

In [658]:
# Model for IP
xgb_model_ip = best_model
xgb_model_ip.fit(X_train, y_train)
xgb_pred_ip = xgb_model_ip.predict(X_valid)

In [659]:
print("Accuracy of Our Model is:")
accuracy(y_valid_ip, xgb_pred_ip)
print("Accuracy of IP_fans is:")
accuracy(starter_train_df["IP"], starter_train_df["IP_fans"])
print("Accuracy of IP_marcel is:")
accuracy(starter_train_df["IP"], starter_train_df["IP_marcel"])
print("Accuracy of IP_rotochamp is:")
accuracy(starter_train_df["IP"], starter_train_df["IP_rotochamp"])
print("Accuracy of IP_zips is:")
accuracy(starter_train_df["IP"], starter_train_df["IP_zips"])

Accuracy of Our Model is:


RMSE,MAE,R^2,RMSLE
54.889366,40.279979,0.300342,0.560175


Accuracy of IP_fans is:


RMSE,MAE,R^2,RMSLE
66.310400,47.778711,-0.084536,0.765327


Accuracy of IP_marcel is:


RMSE,MAE,R^2,RMSLE
61.653084,49.128852,0.062459,0.720947


Accuracy of IP_rotochamp is:


RMSE,MAE,R^2,RMSLE
65.816498,47.652661,-0.068440,0.756959


Accuracy of IP_zips is:


RMSE,MAE,R^2,RMSLE
61.989922,47.754902,0.052187,0.738858


In [668]:
# Model for RA9
xgb_model_ra9 = best_model
xgb_model_ra9.fit(X_train, y_train)
xgb_pred_ra9 = xgb_model_ra9.predict(X_valid)

In [669]:
print("Accuracy of Our Model is:")
accuracy(y_valid_ra9, xgb_pred_ra9)
print("Accuracy of RA9_fans is:")
accuracy(starter_train_df["RA9"], starter_train_df["RA9_fans"])
print("Accuracy of RA9_marcel is:")
accuracy(starter_train_df["RA9"], starter_train_df["RA9_marcel"])
print("Accuracy of RA9_rotochamp is:")
accuracy(starter_train_df["RA9"], starter_train_df["RA9_rotochamp"])
print("Accuracy of RA9_zips is:")
accuracy(starter_train_df["RA9"], starter_train_df["RA9_zips"])

Accuracy of Our Model is:


RMSE,MAE,R^2,RMSLE
0.893631,0.686655,0.310909,0.169381


Accuracy of RA9_fans is:


RMSE,MAE,R^2,RMSLE
1.209965,0.893190,0.037649,0.209704


Accuracy of RA9_marcel is:


RMSE,MAE,R^2,RMSLE
1.236892,0.888536,-0.005661,0.215211


Accuracy of RA9_rotochamp is:


RMSE,MAE,R^2,RMSLE
1.199901,0.884432,0.053592,0.210982


Accuracy of RA9_zips is:


RMSE,MAE,R^2,RMSLE
1.204295,0.879007,0.046648,0.212011


## 2013 Predictions

In [366]:
# We've also got the real 2013 data to see how we did
Actual13_df = pd.read_csv("../Shared Data (Read Only)/Sabermetrics_SPF/test_saber_realization.csv")

In [429]:
free_agents = ["Andy Pettitte",
               "Bartolo Colon",
               "Brandon McCarthy",
               "Francisco Liriano",
               "Hiroki Kuroda",
               "Joe Blanton",
               "Joe Saunders",
               "Kevin Correia",
               "Paul Maholm",
               "Ryan Dempster",
               "Shaun Marcum",
               "Anibal Sanchez",
               "Brett Myers",
               "Hisashi Iwakuma",
               "Jeremy Guthrie",
               "Zack Greinke"]

In [431]:
free_agent_df = Actual13_df.loc[Actual13_df["NAME"].isin(free_agents)]

In [693]:
display(free_agent_df)

,NAME,IP_fans,RA9_fans,IP_marcel,RA9_marcel,IP_rotochamp,RA9_rotochamp,IP_zips,RA9_zips,playerID,...,debut,finalGame,retroID,bbrefID,deathDate,birthDate,IP,RA9,IP_pred_2013,RA9_pred_2013
9,Andy Pettitte,145,4.018068,97.7,3.909177,140,3.571616,90.3,4.453631,pettian01,...,1995-04-29,9/28/2013,petta001,pettian01,NaN,1972-06-15,185.333333,4.127698,135.470901,4.273118
10,Anibal Sanchez,197,4.039846,177.7,4.137848,200,3.974512,185.0,4.606078,sanchan01,...,2006-06-25,9/26/2014,sanca004,sanchan01,NaN,1984-02-27,182.000000,2.769231,146.427109,4.734696
14,Bartolo Colon,135,4.322962,151.0,4.214071,120,4.410075,127.0,4.475409,colonba01,...,1997-04-04,9/28/2014,colob001,colonba01,NaN,1973-05-24,190.333333,2.837128,160.619171,4.014204
21,Brandon McCarthy,158,3.680507,132.7,3.843843,130,4.301184,118.7,4.214071,mccarbr01,...,2005-05-22,9/23/2014,mccab001,mccarbr01,NaN,1983-07-07,135.000000,4.733333,154.394836,4.296455
25,Brett Myers,178,4.518965,90.3,4.344740,150,4.181404,178.0,4.627856,myersbr01,...,2002-07-24,4/19/2013,myerb001,myersbr01,NaN,1980-08-17,21.333333,8.015625,159.929016,4.669268
63,Francisco Liriano,163,4.595189,147.7,5.106975,160,5.019863,154.7,4.050735,liriafr01,...,2005-09-05,9/27/2014,lirif001,liriafr01,NaN,1983-10-26,161.000000,3.018634,117.492905,4.956975
73,Hiroki Kuroda,198,3.952734,190.3,3.811176,180,3.647839,186.0,4.638745,kurodhi01,...,2008-04-04,9/25/2014,kuroh001,kurodhi01,NaN,1975-02-10,201.333333,3.531457,149.573486,4.319222
74,Hisashi Iwakuma,179,4.028957,106.3,3.865621,160,3.985401,128.3,4.203182,iwakuhi01,...,2012-04-20,9/26/2014,iwakh001,iwakuhi01,NaN,1981-04-12,219.666667,2.827011,136.670227,4.657415
92,Jeremy Guthrie,196,4.704080,169.0,4.812970,195,4.769414,163.7,4.791192,guthrje01,...,2004-08-28,9/26/2014,guthj001,guthrje01,NaN,1979-04-08,211.666667,4.209449,124.964195,4.847713
98,Joe Blanton,172,4.943639,157.3,5.041641,170,4.497187,130.3,4.889194,blantjo01,...,2004-09-21,9/3/2013,blanj001,blantjo01,NaN,1980-12-11,132.666667,6.512563,125.019501,5.208083


### More data prep

In [684]:
drop_list_real = ['NAME',
            'playerID',
            'nameFirst',
            'nameLast',
            'nameGiven',
            'retroID',
            'bbrefID',
            'GIDP', 
            'debut',
            'birthDate',
            'birthCountry',
            'birthCity',
            'birthDay',
            'birthMonth',
            'finalGame',
            'birthState',
            'deathYear',
            'deathMonth',
            'deathDay',
            'deathCountry',
            'deathCity',
            'deathState',
            'deathDate',
            'yearID', 
            'stint', 
            'teamID',
            'lgID', 
            'bats', 
            'throws',
            'IP_pred_2013', 
            'RA9_pred_2013'
            ]

In [685]:
X_real = free_agent_df.drop(drop_list_real,axis= 1)
X_real.drop(['IP','RA9'],axis= 1,inplace=True) # Drop our DVs

y_real_ip = free_agent_df['IP']
y_real_ra9 = free_agent_df['RA9']

In [686]:
X_real.columns

Index(['IP_fans', 'RA9_fans', 'IP_marcel', 'RA9_marcel', 'IP_rotochamp',
       'RA9_rotochamp', 'IP_zips', 'RA9_zips', 'W', 'L', 'G', 'GS', 'CG',
       'SHO', 'SV', 'IPouts', 'H', 'ER', 'HR', 'BB', 'SO', 'BAOpp', 'ERA',
       'IBB', 'WP', 'HBP', 'BK', 'BFP', 'GF', 'R', 'SH', 'SF', 'birthYear',
       'weight', 'height'],
      dtype='object')

In [450]:
X_real['BFP_fans'] = (X_real['RA9_fans']/9) + (X_real['IP_fans']*3)
X_real['BFP_marcel'] = (X_real['RA9_marcel']/9) + (X_real['IP_marcel']*3)
X_real['BFP_zips'] = (X_real['RA9_zips']/9) + (X_real['IP_zips']*3)
X_real['BFP_rotochamp'] = (X_real['RA9_rotochamp']/9) + (X_real['IP_rotochamp']*3)

In [441]:
res_train = encode_and_bind(X_real, enc_list)

In [442]:
X_real = res_train

In [687]:
X_real.drop(pred_drop_list,axis= 1,inplace=True)

In [688]:
# Our number of columns aren't matching up because there are no Diamondbacks in the training data...
a = X_train.columns.difference(X_real.columns)
print (a)

Index([], dtype='object')


In [454]:
X_real = X_real.sort_index(axis=1)

In [452]:
X_real['bats_B'] = 0
X_real['teamID_BAL'] = 0
X_real['teamID_CHA'] = 0
X_real['teamID_CHN'] = 0
X_real['teamID_CIN'] = 0
X_real['teamID_COL'] = 0
X_real['teamID_HOU'] = 0
X_real['teamID_MIA'] = 0
X_real['teamID_MIL'] = 0
X_real['teamID_PHI'] = 0
X_real['teamID_SDN'] = 0
X_real['teamID_SFN'] = 0
X_real['teamID_SLN'] = 0
X_real['teamID_TBA'] = 0
X_real['teamID_TEX'] = 0
X_real['teamID_TOR'] = 0
X_real['teamID_WAS'] = 0

### Apply the model

In [689]:
xgb_pred_ip = xgb_model_ip.predict(X_real)

ValueError: feature_names mismatch: ['IP_fans', 'IP_marcel', 'IP_rotochamp', 'IP_zips', 'RA9_fans', 'RA9_marcel', 'RA9_rotochamp', 'RA9_zips', 'birthYear', 'height', 'weight'] ['IP_fans', 'RA9_fans', 'IP_marcel', 'RA9_marcel', 'IP_rotochamp', 'RA9_rotochamp', 'IP_zips', 'RA9_zips', 'birthYear', 'weight', 'height']

In [456]:
xgb_pred_ra9 = xgb_model_ra9.predict(X_real)

## Actual Eval

In [ ]:
# How did our model do on the test data?
accuracy(free_agent_df["IP"], free_agent_df["IP_pred_2013"])

RMSE,MAE,R^2,RMSLE
56.695497,44.168303,-0.323380,0.575628


In [ ]:
# How did our model do on the test data?
accuracy(free_agent_df["RA9"], free_agent_df["RA9_pred_2013"])

RMSE,MAE,R^2,RMSLE
1.444829,1.221418,0.031893,0.263052


In [691]:
print("Accuracy of IP_fans is:")
accuracy(free_agent_df["IP"], free_agent_df["IP_fans"])
print("Accuracy of IP_marcel is:")
accuracy(free_agent_df["IP"], free_agent_df["IP_marcel"])
print("Accuracy of IP_rotochamp is:")
accuracy(free_agent_df["IP"], free_agent_df["IP_rotochamp"])
print("Accuracy of IP_zips is:")
accuracy(free_agent_df["IP"], free_agent_df["IP_zips"])

Accuracy of IP_fans is:


RMSE,MAE,R^2,RMSLE
51.816517,36.895833,-0.105411,0.572172


Accuracy of IP_marcel is:


RMSE,MAE,R^2,RMSLE
46.759069,33.800000,0.099841,0.463937


Accuracy of IP_rotochamp is:


RMSE,MAE,R^2,RMSLE
49.129011,37.520833,0.006281,0.541267


Accuracy of IP_zips is:


RMSE,MAE,R^2,RMSLE
61.012549,43.664583,-0.532589,0.614061


In [690]:
print("Accuracy of RA9_fans is:")
accuracy(free_agent_df["RA9"], free_agent_df["RA9_fans"])
print("Accuracy of RA9_marcel is:")
accuracy(free_agent_df["RA9"], free_agent_df["RA9_marcel"])
print("Accuracy of RA9_rotochamp is:")
accuracy(free_agent_df["RA9"], free_agent_df["RA9_rotochamp"])
print("Accuracy of RA9_zips is:")
accuracy(free_agent_df["RA9"], free_agent_df["RA9_zips"])

Accuracy of RA9_fans is:


RMSE,MAE,R^2,RMSLE
1.366752,1.149506,0.133697,0.240359


Accuracy of RA9_marcel is:


RMSE,MAE,R^2,RMSLE
1.435602,1.184215,0.044220,0.254088


Accuracy of RA9_rotochamp is:


RMSE,MAE,R^2,RMSLE
1.490604,1.215521,-0.030421,0.262097


Accuracy of RA9_zips is:


RMSE,MAE,R^2,RMSLE
1.421971,1.194331,0.062283,0.251248


## Exporting for Reporting

In [468]:
free_agent_df['IP_pred_2013'] = xgb_pred_ip

In [469]:
free_agent_df['RA9_pred_2013'] = xgb_pred_ra9

In [475]:
free_agent_df.to_csv('free_agent_preds.csv')